---
## 🎯 Practice Exercises

### Exercise 1: Adaptive Reflection with Quality Metrics

**Task:** Improve Reflection with numerical quality scoring.

**Requirements:**
1. Critic scores draft on multiple criteria (1-5 each):
   - Clarity
   - Completeness
   - Accuracy
2. Use Pydantic model for structured scoring
3. Only refine if any score < 4
4. Track score improvements across iterations
5. Stop when all scores ≥ 4 or max iterations

**Example:**
```
Iteration 1: Clarity=3, Completeness=4, Accuracy=5 → Refine
Iteration 2: Clarity=5, Completeness=4, Accuracy=5 → Approved!
```

**Deliverables:**
- Reflection agent with quality metrics
- Test with task requiring refinement
- Visualize score improvements

In [17]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import operator
import os
import json


print("✅ All imports successful")

✅ All imports successful


In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("✅ API key loaded")

✅ API key loaded


In [3]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [11]:
def generator(state):
    pass

generator


<function __main__.generator(state)>

In [4]:
from pydantic import BaseModel, Field

class QualityScore(BaseModel):
    clarity: int = Field(..., ge=1, le=5)
    completeness: int = Field(..., ge=1, le=5)
    accuracy: int = Field(..., ge=1, le=5)


In [5]:
class ReflectionState(TypedDict):
    task: str
    draft: str
    critique: str
    iterations: int
    final_output: str
    scores: list[QualityScore]  # Track scores over iterations


In [18]:
def critic(state: ReflectionState) -> dict:
    """Evaluate draft with numerical quality scores."""
    
    prompt = f"""
Evaluate the response below on a scale of 1–5 for:
- Clarity
- Completeness
- Accuracy

Return STRICT JSON:
{{
  "clarity": int,
  "completeness": int,
  "accuracy": int,
  "critique": "text feedback"
}}

Task: {state['task']}
Response: {state['draft']}
"""
    print("🔍 Scoring draft...")
    response = llm.invoke([HumanMessage(content=prompt)])
    
    raw = json.loads(response.content)
    score = QualityScore(
        clarity=raw["clarity"],
        completeness=raw["completeness"],
        accuracy=raw["accuracy"]
    )
    
    print(
        f"Scores → "
        f"Clarity={score.clarity}, "
        f"Completeness={score.completeness}, "
        f"Accuracy={score.accuracy}"
    )

    return {
        "critique": raw["critique"],
        "iterations": state["iterations"] + 1,
        "scores": state.get("scores", []) + [score]
    }


In [19]:
MAX_REFLECTIONS = 3

def should_reflect_again(state: ReflectionState) -> Literal["generator", "finalizer"]:
    last_score = state["scores"][-1]

    if (
        last_score.clarity >= 4
        and last_score.completeness >= 4
        and last_score.accuracy >= 4
    ):
        print("✅ Quality threshold met — approved!")
        return "finalizer"

    if state["iterations"] >= MAX_REFLECTIONS:
        print("⚠️ Max iterations reached")
        return "finalizer"

    print("🔁 Scores below threshold — refining...")
    return "generator"


In [20]:
def reflection_finalizer(state: ReflectionState) -> dict:
    print("\n🎉 Reflection complete!")
    return {"final_output": state["draft"]}


In [21]:
reflection_builder = StateGraph(ReflectionState)

reflection_builder.add_node("generator", generator)
reflection_builder.add_node("critic", critic)
reflection_builder.add_node("finalizer", reflection_finalizer)

reflection_builder.add_edge(START, "generator")
reflection_builder.add_edge("generator", "critic")
reflection_builder.add_conditional_edges(
    "critic",
    should_reflect_again,
    {"generator": "generator", "finalizer": "finalizer"}
)
reflection_builder.add_edge("finalizer", END)

reflection_agent = reflection_builder.compile()


In [22]:
result = reflection_agent.invoke({
    "task": "Explain what an agentic pattern is in 2–3 sentences",
    "draft": "",
    "critique": "",
    "iterations": 0,
    "scores": []
})


🔍 Scoring draft...
Scores → Clarity=4, Completeness=3, Accuracy=5
🔁 Scores below threshold — refining...
🔍 Scoring draft...
Scores → Clarity=4, Completeness=3, Accuracy=5
🔁 Scores below threshold — refining...
🔍 Scoring draft...
Scores → Clarity=4, Completeness=3, Accuracy=5
⚠️ Max iterations reached

🎉 Reflection complete!


In [23]:
print("\n📈 SCORE IMPROVEMENT TRACKING")
print("=" * 50)

for i, score in enumerate(result["scores"], start=1):
    print(
        f"Iteration {i}: "
        f"Clarity={score.clarity}, "
        f"Completeness={score.completeness}, "
        f"Accuracy={score.accuracy}"
    )

print("=" * 50)
print("✅ Final Output:")
print(result["final_output"])



📈 SCORE IMPROVEMENT TRACKING
Iteration 1: Clarity=4, Completeness=3, Accuracy=5
Iteration 2: Clarity=4, Completeness=3, Accuracy=5
Iteration 3: Clarity=4, Completeness=3, Accuracy=5
✅ Final Output:

